# Multinetwork storage optimization with PandaModels.jl
This tutorial describes how to run a storage optimization over multiple timesteps with a PandaModels.jl multinetwork
together with pandapower.

To run a storage optimization over multiple time steps, the power system data is copied n_timestep times internally.
This is done efficiently in a julia script. Each network in the multinetwork dict represents a single time step. 
The input time series must be written to the loads and generators accordingly to each network. 
This is currently done by converting input time series to pandapwower controllers, saving it together with the grid data as a json file and loading the data back in julia. This "hack" is probably just a temporary solution. 

Some notes:
* only storages which are set as "controllable" are optimized
* time series can be written to load / sgen elements
* output of the optimization is a dict containing pandas DataFrames for every optimized storage and time step   

For more details on PowerModels (PandaModels) storage model see:

https://lanl-ansi.github.io/PowerModels.jl/stable/storage/ and 
https://github.com/e2nIEE/PandaModels.jl/blob/develop/src/models/call_powermodels.jl

For more details on PowerModels multinetworks see:

https://lanl-ansi.github.io/PowerModels.jl/stable/multi-networks/


# Installation
You need the standard Julia, PowerModels, Ipopt and JuMP Installation (see the opf_powermodels.ipynb).
 

# Run the storage optimization
In order to start the optimization and visualize results, we follow four steps:
1) Load the pandapower grid data (here the cigre MV grid)
2) Convert the time series to pandapwoer-controllers
3) Start the optimization
4) Get and plot the results


## 1) Get the grid data
We load the cigre medium voltage grid with "pv" and "wind" generators. Also we set some limits and add a storage with
**controllable** == True


In [13]:
import pandapower as pp
import pandapower.networks as nw

def cigre_grid():
    net = nw.create_cigre_network_mv("pv_wind")
    # set some limits
    min_vm_pu = 0.95
    max_vm_pu = 1.05

    net["bus"].loc[:, "min_vm_pu"] = min_vm_pu
    net["bus"].loc[:, "max_vm_pu"] = max_vm_pu

    net["line"].loc[:, "max_loading_percent"] = 100.

    # close all switches
    net.switch.loc[:, "closed"] = True
    # add storage to bus 10
    pp.create_storage(net, 10, p_mw=0.5, max_e_mwh=.2, soc_percent=0., q_mvar=0., controllable=True)

    return net



## 2) Convert the time series to pandapower-controllers
The following functions loads the example time series from the input_file and scales the power accordingly.
It then adds the time series data to the grid model by creating controllers.


In [14]:
import pandas as pd
from pandapower.control import ConstControl
from pandapower.timeseries import DFData

def convert_timeseries_to_controller(net, input_file):
    
    # set the load type in the cigre grid, since it is not specified
    net["load"].loc[:, "type"] = "residential"
    
    # set the sgen type in the cigre grid
    net.sgen.loc[:, "type"] = "pv"
    net.sgen.loc[8, "type"] = "wind"

    # read the example time series
    time_series = pd.read_json(input_file)
    time_series.sort_index(inplace=True)

    # this example time series has a 15min resolution with 96 time steps for one day
    n_timesteps = time_series.shape[0]
    
    # get rated power
    load_p = net["load"].loc[:, "p_mw"].values
    sgen_p = net["sgen"].loc[:7, "p_mw"].values
    wind_p = net["sgen"].loc[8, "p_mw"]

    load_ts = pd.DataFrame(index=time_series.index.tolist(), columns=net.load.index.tolist())
    sgen_ts = pd.DataFrame(index=time_series.index.tolist(), columns=net.sgen.index.tolist())
    for t in range(n_timesteps):
        load_ts.loc[t] = load_p * time_series.at[t, "residential"]
        sgen_ts.loc[t][:8] = sgen_p * time_series.at[t, "pv"]
        sgen_ts.loc[t][8] = wind_p * time_series.at[t, "wind"]

    # create time series controller for load and sgen 
    ConstControl(net, element="load", variable="p_mw",
                 element_index=net.load.index.tolist(), profile_name=net.load.index.tolist(),
                 data_source=DFData(load_ts))
    ConstControl(net, element="sgen", variable="p_mw",
                 element_index=net.sgen.index.tolist(), profile_name=net.sgen.index.tolist(),
                 data_source=DFData(sgen_ts))


## 3) Start the optimization 
Before we start the optimization, we create the grid and controller, adding the time series in 15min resolution. 


In [15]:
# open the cigre mv grid
net = cigre_grid()

# convert the time series to pandapower controller
input_file = "cigre_timeseries_15min.json"
convert_timeseries_to_controller(net, input_file)


In [16]:
net = cigre_grid()
input_file = "cigre_timeseries_15min.json"

    
# set the load type in the cigre grid, since it is not specified
net["load"].loc[:, "type"] = "residential"

# set the sgen type in the cigre grid
net.sgen.loc[:, "type"] = "pv"
net.sgen.loc[8, "type"] = "wind"

# read the example time series
time_series = pd.read_json(input_file)
time_series.sort_index(inplace=True)

# this example time series has a 15min resolution with 96 time steps for one day
n_timesteps = time_series.shape[0]

# get rated power
load_p = net["load"].loc[:, "p_mw"].values
sgen_p = net["sgen"].loc[:7, "p_mw"].values
wind_p = net["sgen"].loc[8, "p_mw"]

load_ts = pd.DataFrame(index=time_series.index.tolist(), columns=net.load.index.tolist())
sgen_ts = pd.DataFrame(index=time_series.index.tolist(), columns=net.sgen.index.tolist())

n_timesteps
sgen_ts.loc[10][8]

#for t in range(n_timesteps):
#    load_ts.loc[t] = load_p * time_series.at[t, "residential"]
#    sgen_ts.loc[t][:8] = sgen_p * time_series.at[t, "pv"]
#    sgen_ts.loc[t][8] = wind_p * time_series.at[t, "wind"]

# create time series controller for load and sgen 
#ConstControl(net, element="load", variable="p_mw",
#                element_index=net.load.index.tolist(), profile_name=net.load.index.tolist(),
#                data_source=DFData(load_ts))
#ConstControl(net, element="sgen", variable="p_mw",
#                element_index=net.sgen.index.tolist(), profile_name=net.sgen.index.tolist(),
#                data_source=DFData(sgen_ts))

nan

Now, the time series is added through (const) controllers, and you can check the created controllers


In [17]:
# print controller
print("--- time series controller:", net.controller)

# print time series data in controller
print("--- considered element of controller 0:", net.controller.object[0].__dict__["matching_params"]["element"])
print("--- considered element index of controller 0:",net.controller.object[0].__dict__["matching_params"]["element_index"])
print("--- time series data:",net.controller.object[0].data_source.df)


--- time series controller: Empty DataFrame
Columns: [object, in_service, order, level, initial_run, recycle]
Index: []


KeyError: 0

We start the optimization for timesteps from 0 to 10.


In [ ]:
# run the optimization for the first ten timesteps (the first run can be slow.
try:
    pp.runpm_storage_opf(net, from_time_step=0, to_time_step=10)
except Exception as err:
    print(err)

no costs are given - overall generated power is minimized


## 4) Get and plot the results 
Get and plot the optimization results for the storage.


In [ ]:
from pandapower.opf.pm_storage import read_pm_storage_results
import matplotlib.pyplot as plt

def plot_storage_results(storage_results):
    n_res = len(storage_results.keys())
    fig, axes = plt.subplots(n_res, 2)
    if n_res == 1:
        axes = [axes]
    for i, (key, val) in enumerate(storage_results.items()):
        res = val
        axes[i][0].set_title("Storage {}".format(key))
        el = res.loc[:, ["p_mw", "q_mvar", "soc_mwh"]]
        el.plot(ax=axes[i][0])
        axes[i][0].set_xlabel("time step")
        axes[i][0].legend(loc=4)
        axes[i][0].grid()
        ax2 = axes[i][1]
        patch = plt.plot([], [], ms=8, ls="--", mec=None, color="grey", label="{:s}".format("soc_percent"))
        ax2.legend(handles=patch)
        ax2.set_label("SOC percent")
        res.loc[:, "soc_percent"].plot(ax=ax2, linestyle="--", color="grey")
        ax2.grid()
    plt.show()

# get the results
#storage_results = read_pm_storage_results(net)    
    
# plot the results
#plot_storage_results(storage_results)